In [1]:
import torch, struct, os
import numpy as np
import tensorflow as tf

2024-01-31 14:42:36.281353: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-01-31 14:42:36.314137: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-31 14:42:36.314167: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-31 14:42:36.314852: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-31 14:42:36.319592: I tensorflow/core/platform/cpu_feature_guar

In [5]:
# TODO: use json file for model (results, config etc) management
model_name = "dnn_mnist_pt"
model_path = "../../models/"

state_dict = torch.load(model_path + model_name + ".pth", map_location=torch.device("cpu"))
list_vars = state_dict

In [6]:
folder = "./tmp/"

# Create the directory 'tmp' in the current working directory
try:
    os.makedirs(folder, exist_ok=True)
    print(f"Directory '{folder}' created successfully")
except OSError as error:
    print(f"Directory '{folder}' cannot be created. Error: {error}")

Directory './tmp/' created successfully


In [16]:
fname_out = folder + "ggml-model-" + model_name + ".bin"
pack_fmt = "!i"


fout = open(fname_out, "w+b")
fout.write(struct.pack(pack_fmt, 0x67676d6c)) # magic: ggml in hex

for name in list_vars.keys():
    data = list_vars[name].squeeze().numpy()
    print("Processing variable: " + name + " with shape: ", data.shape) 
    n_dims = len(data.shape)
   
    fout.write(struct.pack(pack_fmt, n_dims))
    
    data = data.astype(np.float32)
    for i in range(n_dims):
        fout.write(struct.pack(pack_fmt, data.shape[n_dims - 1 - i]))

    # data
    data = data.astype(">f4")
    data.tofile(fout)

fout.close()

Processing variable: fc1.weight with shape:  (56, 784)
Processing variable: fc1.bias with shape:  (56,)
Processing variable: fc2.weight with shape:  (10, 56)
Processing variable: fc2.bias with shape:  (10,)


In [7]:
# Load TensorFlow MNIST data
mnist = tf.keras.datasets.mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

In [17]:
def load_img_from_file(data_file="input", show=False):
    try:
        with open(data_file, 'rb') as file:
            buf = file.read()
    except Exception as e:
        print(e)
        return None, e

    digits = np.frombuffer(buf, dtype=np.uint8).astype(np.float32)
    
    if show:
        c = ""
        for row in range(28):
            for col in range(28):
                if buf[row * 28 + col] > 230:
                    c += "&"
                else:
                    c += "-"
            c += "\n"
        print(c)

    return digits, None

def save_img_to_file(image, data_file = "input"):
    try:
        # Convert to bytes
        image_bytes = image.astype('uint8').tobytes()
        
        # Write to file
        with open(data_file, 'wb') as file:
            file.write(image_bytes)
        
        print(f"Image saved to {data_file}")
    except Exception as e:
        print(f"Error saving image: {e}")


In [57]:
index = 9
img_out_path = folder + str(index) 
save_img_to_file(test_images[index], img_out_path)

Image saved to ./tmp/9


In [58]:
# Example usage
digits, error = load_img_from_file(img_out_path,show=True)

----------------------------
----------------------------
----------------------------
----------------------------
----------------------------
----------------------------
----------------------------
----------------------------
-------------&&&&&&&--------
-----------&&&---&-&--------
---------&&&--------&-&-----
--------&&&---------&&&-----
-------&&----------&&&------
-------&&-------&&&&&&------
--------&&&&&&&&&&&&--------
---------------&&&&---------
---------------&&&----------
--------------&&&-----------
-------------&&&------------
------------&&&&------------
------------&&&-------------
-----------&&&--------------
-----------&&---------------
----------&&&---------------
----------&&----------------
----------&&----------------
----------&-----------------
----------------------------



## Test

In [74]:
import subprocess
import psutil
import threading
import time

# Function to get current memory usage
def get_memory_usage():
    return psutil.Process(os.getpid()).memory_info().rss / (1024 * 1024)  # Memory in MB

def monitor_memory(pid):
    p = psutil.Process(pid)
    max_memory = 0
    while True:
        try:
            mem = p.memory_info().rss / (1024 * 1024)
            max_memory = max(max_memory, mem)
        except psutil.NoSuchProcess:
            break  # Process has finished
        time.sleep(0.01)  # Poll every second

    print(f"Maximum memory used: {max_memory} MB")

In [75]:
vm_file = "./bin/vm"
program = "./bin/gglm.bin"
model_in_path = fname_out
#input_img = data_file

In [76]:
# start_memory = get_memory_usage()
# start_time = time.time()

command = [f"{vm_file}", f"--basedir={folder}",
                f"--program={program}", f"--model={model_in_path}", 
                f"--data={img_out_path}", "--mipsVMCompatible"]

process = subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)

# Get the process ID
pid = process.pid
print(f"Process ID: {pid}")


p = psutil.Process(pid)


# Start memory monitoring in a separate thread
monitor_thread = threading.Thread(target=monitor_memory, args=(pid,))
monitor_thread.start()

# Wait for the process to complete and capture output
stdout, stderr = process.communicate()

# Wait for the monitoring thread to finish
monitor_thread.join()

Process ID: 450291
Maximum memory used: 87.6015625 MB


In [63]:
stderr[-2]

'9'